# LinearModel

In [ ]:
import SparseArrays: sparse
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("EnsembleInputs.ipynb")
@nbinclude("SuppressImplicit.ipynb");

## Training

In [ ]:
function save_linear_model(alphas::Vector{String}, content::String, outdir::String)
    if content == "explicit"
        implicit = false
    elseif content in ["implicit", "ptw"]
        implicit = true
    else
        @assert false
    end
    set_logging_outdir(outdir)
    _, β = regress(alphas, content, implicit)
    @info "alphas: $alphas"
    @info "coefficients: $β"
    write_params(Dict("β" => β, "alphas" => alphas), outdir)

    function model(split::String, content::String; raw_splits::Bool)
        read_fn = raw_splits ? read_raw_alpha : read_alpha
        Xs = [read_fn(alpha, split, content).rating for alpha in alphas]
        if implicit
            push!(Xs, fill(one(Float32), length(Xs[1])))
        end
        X = hcat(Xs...)
        y = X * β
        vec(y)
    end
    write_alpha(
        model,
        outdir;
        by_split = true,
        log = true,
        log_implicit = implicit,
        log_alphas = String[],
        log_content = content,
        log_splits = ["validation", "test"],
    )
end;

In [ ]:
save_linear_model(explicit_raw_alphas, "explicit", "LinearExplicit");

In [ ]:
save_linear_model(implicit_raw_alphas, "implicit", "LinearImplicit");

In [ ]:
save_linear_model(ptw_raw_alphas, "ptw", "LinearPtw");